In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/val.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test_without_labels.csv')

In [ ]:
# get labels
train_label_raw = list(train_df.labels)
val_label_raw = list(val_df.labels)

In [ ]:
train_labels = []
for label in train_label_raw:
    train_labels.append(label.split(" "))

val_labels = []
for label in val_label_raw:
    val_labels.append(label.split(" "))

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer

def get_lemmatized_tokens(dataframe):
  """
  Extract sentences from dataframe and output stemmed tokens and related NER
  Extract labels from dataframe and tokenize it
  """
  sentences = list(dataframe.sents)

  tokenized_sentences = []
  for sentence in sentences:
    tokens = sentence.split(" ")
    tokenized_sentences.append(tokens)

  lemmatized_sentences = []
  lemmatizer = WordNetLemmatizer()

  for sentence in tokenized_sentences:
    lemmatized = [lemmatizer.lemmatize(word, pos='v').lower() for word in sentence]
    lemmatized_sentences.append(lemmatized)

  return lemmatized_sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
train_tokens = get_lemmatized_tokens(train_df)
val_tokens = get_lemmatized_tokens(val_df)
test_tokens = get_lemmatized_tokens(test_df)

In [ ]:
all_tokens = train_tokens + val_tokens + test_tokens

In [ ]:
word_to_ix = {}
for sentence in all_tokens:
    for word in sentence:
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

word_list = list(word_to_ix.keys())

START_TAG = "<START>"
STOP_TAG = "<STOP>"

tag_to_ix = {START_TAG:0, STOP_TAG:1}

for tags in train_labels:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

In [ ]:
nltk.download('averaged_perceptron_tagger')
def generate_pos_tag(doc):
  """
  :param doc_words: words in doc
  :return: pos tags
  """
  pos_tags = []
  for sentence in doc:
    tags = []
    for word, tag in nltk.pos_tag(sentence):
      tags.append(tag)
    pos_tags.append(tags)
 
  return pos_tags

pos_tags = generate_pos_tag(all_tokens)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# use word2vec to train the embeddings for pos
from gensim.models import Word2Vec

w2v_pos = Word2Vec(sentences=pos_tags, 
                   size=20, 
                   window=3,
                   min_count=1,
                   workers=4,
                   sg=1)

In [ ]:
#Lab 7 
import spacy

#load the spacy api with the pre-trained statistical models for English. English multi-task CNN trained on OntoNotes
nlp = spacy.load("en_core_web_sm")

def parse_sentence(data):
    parse_sentences_temp=[]
    for sentence in data:
        parse = nlp(' '.join(sentence))
        deps = []
        for x in parse:
          deps.append(x.dep_)

        parse_sentences_temp.append(deps[:len(sentence)])
    return parse_sentences_temp


In [ ]:
parse_sentences = parse_sentence(all_tokens)

In [ ]:
parse_sentences[0]

['compound',
 'compound',
 'nsubj',
 'punct',
 'parataxis',
 'punct',
 'punct',
 'nmod',
 'intj',
 'intj',
 'punct',
 'nmod',
 'intj',
 'compound',
 'compound',
 'compound',
 'nmod',
 'intj',
 'compound',
 'compound',
 'compound',
 'compound',
 'appos',
 'punct',
 'ROOT',
 'det',
 'amod',
 'attr',
 'attr',
 'agent',
 'compound',
 'pobj',
 'prep',
 'amod',
 'pobj',
 'nummod',
 'aux',
 'advcl',
 'det',
 'dobj',
 'prep',
 'amod',
 'pobj',
 'advcl',
 'det',
 'dobj',
 'cc',
 'conj',
 'det',
 'amod',
 'dobj',
 'punct']

In [ ]:
from gensim.models import Word2Vec
w2v_parse_tree = Word2Vec(sentences=parse_sentences, 
                   size=20, 
                   window=3,
                   min_count=1,
                   workers=4,
                   sg=1)

In [ ]:
word_2_pos={}
for i in range(0,len(all_tokens)):
    for x in range(0,len(all_tokens[i])):
        word_2_pos[all_tokens[i][x]] = w2v_pos[pos_tags[i][x]]

word_2_pt={}
for i in range(0,len(all_tokens)):
    for x in range(0,len(all_tokens[i])):
        word_2_pt[all_tokens[i][x]] = w2v_parse_tree[parse_sentences[i][x]]  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [ ]:
import gensim.downloader as api
import numpy as np
word_emb_model = api.load("glove-twitter-100") 

[==================================================] 100.0% 387.1/387.1MB downloaded


In [ ]:
# 100 w2v + 20 pos + 20 tree
EMBEDDING_DIM = 140
embedding_matrix = []
for word in word_list:
    try:
        a = word_emb_model.wv[word]
        b = np.append(a, word_2_pos[word])
        c = np.append(b, word_2_pt[word])
        
        # 加到这里 - 向量化
        # word_embedding_temp.extend(len(word))
        embedding_matrix.append(c)
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)

embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


(3408, 140)

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(train_tokens,word_to_ix)
train_output_index = to_index(train_labels,tag_to_ix)
val_input_index = to_index(val_tokens,word_to_ix)
val_output_index = to_index(val_labels,tag_to_ix)
test_input_index = to_index(test_tokens,word_to_ix)

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))



## stacked bi-lstm layer >= 1
## attention - 至少三种 attention
## CRF layer, 有或者，没有


class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, layers):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        
        self.tagset_size = len(tag_to_ix)
      

        self.layers = layers
        
                                                                          ## batch_size * dim1 * dim2
        self.general_attention_weight = nn.parameter.Parameter(torch.Tensor(1, self.hidden_dim, self.hidden_dim), requires_grad = True)
        
       


        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        self.dropout = nn.Dropout(0.2)
        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim * 2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()
        self.attn = torch.nn.Linear(self.hidden_dim, self.hidden_dim)

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))
      

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        ## attention part    
    
        query = lstm_out.view(lstm_out.size(1), lstm_out.size(0), lstm_out.size(2))
        
        key = lstm_out.view(lstm_out.size(1), lstm_out.size(2), lstm_out.size(0))
        
        value = lstm_out.view(lstm_out.size(1), lstm_out.size(0), lstm_out.size(2))
        
        weight_att = nn.functional.softmax(torch.bmm(query, key),dim=-1)
        
      

        output = torch.bmm(weight_att, value)
        concat_output = torch.cat((output, value), dim = -1)
        lstm_out = concat_output.view(len(sentence), self.hidden_dim * 2)
        

        lstm_out = self.dropout(lstm_out)
        lstm_feats = self.hidden2tag(lstm_out)
      
        return lstm_feats

    def _score_sentence(self, feats, tags):
        
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        
        return forward_score - gold_score

    def forward(self, sentence):
        
        # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
          
        lstm_feats = self._get_lstm_features(sentence)
        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        
        return score, tag_seq

In [ ]:
import numpy as np
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    return predicted, ground_truth, accuracy

In [ ]:
device = torch.device("cpu")
HIDDEN_DIM = 50


model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, layers = 1).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005, weight_decay=1e-4)

In [ ]:
train_data=train_input_index+val_input_index
train_label=train_output_index+val_output_index

In [ ]:
"""Each epoch will take about 1-2 minutes"""
import datetime
attention_train_loss = []
attention_val_loss = []
for epoch in range(20):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    
    for i, idxs in enumerate(train_data):

        tags_index = train_label[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        
        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)
        
        

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()
    attention_train_loss.append(train_loss)
    max_val_acc=0
    model.eval()
    _, _, train_acc = cal_acc(model, train_input_index, train_output_index)
    _, _, val_acc = cal_acc(model, val_input_index, val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    attention_val_loss.append(val_loss)
    time2 = datetime.datetime.now()
    print("Epoch:%d, Training loss: %.2f, train_acc: %.4f, val loss: %.2f, val_acc: %.4f, time: %.2fs" %(epoch+1, train_loss, train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

Epoch:1, Training loss: 22154.86, train_acc: 0.7398, val loss: 4573.85, val_acc: 0.6942, time: 120.94s
Epoch:2, Training loss: 13507.29, train_acc: 0.7841, val loss: 3228.08, val_acc: 0.7429, time: 122.17s
Epoch:3, Training loss: 10464.66, train_acc: 0.8229, val loss: 2540.91, val_acc: 0.7975, time: 121.81s
Epoch:4, Training loss: 8771.06, train_acc: 0.8329, val loss: 2150.33, val_acc: 0.8072, time: 121.87s
Epoch:5, Training loss: 7625.51, train_acc: 0.8405, val loss: 1881.04, val_acc: 0.8140, time: 122.26s
Epoch:6, Training loss: 6808.34, train_acc: 0.8510, val loss: 1637.95, val_acc: 0.8385, time: 122.60s
Epoch:7, Training loss: 6089.71, train_acc: 0.8646, val loss: 1490.22, val_acc: 0.8489, time: 122.62s
Epoch:8, Training loss: 5667.21, train_acc: 0.8699, val loss: 1331.14, val_acc: 0.8667, time: 122.63s
Epoch:9, Training loss: 5268.89, train_acc: 0.8745, val loss: 1228.78, val_acc: 0.8739, time: 123.29s
Epoch:10, Training loss: 4874.40, train_acc: 0.8918, val loss: 1124.07, val_acc

In [ ]:
torch.save(model, "best_model.pth")

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
model_best = torch.load('97% model.pth')

In [ ]:
ix_2_tag = {idx : tag for tag, idx in tag_to_ix.items()}

In [ ]:
device = torch.device("cpu")
res = []
for x in test_input_index:
  input_tensor = torch.tensor(x).to(device)
  _, output = model_best(input_tensor)

  for i in output:
    res.append(ix_2_tag[i])

In [ ]:
import pandas as pd
import numpy as np
data_re=pd.core.frame.DataFrame({'Id':range(len(res)) ,'Predicted':res })
data_re.to_csv('result.csv', index=False)

reference ：
1. Vaswani, A., Shazeer, N.M., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A.N., Kaiser, L., & Polosukhin, I.. Attention is All you Need. ArXiv, abs/1706.03762. (2017).
2. Zhou, P., Shi, W., Tian, J., Qi, Z., Li, B., Hao, H., & Xu, B.. Attention-based bidirectional long short-term memory networks for relation classification.  in Proceedings of the 54th Annual Meeting of the Association for Computational Linguistics (Volume 2: Short Papers) pp. 207–212.  (2016).
3. lab 7, lab 9, lab 10
4. SSBakhSSBakh                    1, et al. “How to Use pos_tag in NLTK?” Stack Overflow, https://stackoverflow.com/questions/47519987/how-to-use-pos-tag-in-nltk. (n.d.)


5. Gensim: topic modelling for humans. models.word2vec – Word2vec embeddings - gensim. https://radimrehurek.com/gensim/models/word2vec.html.  (2021, April 29)
